<p align="center" ><img src="https://www.ai4kids.ai/wp-content/uploads/2019/07/ai4kids_website_logo_120x40.png"></img></p>

# AI 專題實作：貓狗辨識

貓狗辨識專題使用的是 **限時塗鴉 (Quick, Draw!)** 提供的貓與狗資料集。官方提供 4 種不同的資料格式，在本專題實作中將使用的是 NumPy 陣列 (npy)。

- 資料集說明可以參考官方 GitHub：[The Quick, Draw! Dataset](https://github.com/googlecreativelab/quickdraw-dataset)

- 限時塗鴉體驗網站：[https://quickdraw.withgoogle.com/](https://quickdraw.withgoogle.com/)

<p align="right">© Copyright AI4kids.ai</p>

## 1. 下載資料集

Google 將開放的限時塗鴉資料集存放在 Google Cloud Platform 的 Storage，從網址 [https://console.cloud.google.com/storage/browser/quickdraw_dataset/full](https://console.cloud.google.com/storage/browser/quickdraw_dataset/full) 可以瀏覽所有資料集。

在專題中，執行下列指令直接下載，並儲存在 Colab 中。

In [ ]:
import urllib.request

urllib.request.urlretrieve('https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cat.npy', 'cat.npy')
urllib.request.urlretrieve('https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/dog.npy', 'dog.npy')

瀏覽已下載檔案

In [ ]:
!ls -l

## 2. 載入資料集與資料預處理

載入專題會用到的套件。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.utils import to_categorical

In d:\tf-env\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In d:\tf-env\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In d:\tf-env\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


將貓、狗的資料集載入至 NumPy 陣列。

In [2]:
dog = np.load('dog.npy', encoding='bytes', allow_pickle=True)
cat = np.load('cat.npy', encoding='bytes', allow_pickle=True)

ValueError: cannot reshape array of size 63356848 into shape (152159,784)

載入後資料型別為 NumPy 陣列，查看 `shape` 屬性可以看到 dog 與 cat 分別有 152159 與 123202 筆資料 (也就是有 152159 與 123202 張圖片)。

原始圖片的大小為 28x28，在資料集中將其攤平為一維的維度，所以每筆共有 784 個資料點。

In [ ]:
dog.shape

In [ ]:
cat.shape

使用 Matplotlib 來將陣列資料顯示為原始圖檔，在顯示之前先將資料點維度回復成為二維 28x28 的陣列。

可以看到原始圖檔如下。

In [ ]:
fig=plt.figure(figsize=(10, 10))
columns = 5
rows = 1

for i in range(1, columns * rows + 1):
    
    img = dog[i].reshape(28,28)
    
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)

plt.show()

在專題中，我們僅取狗與貓資料的前 60000 筆來進行訓練及測試。

In [ ]:
sample_size = 60000

將狗與貓資料合併，成為同一個輸入資料。

In [ ]:
X = np.concatenate((dog[:sample_size], cat[:sample_size]))

將資料集資料重塑為原始圖檔的維度，並且將各個元素值除以 255.0 進行**正規化 (Normalization)**處理。

In [ ]:
X = X.reshape(-1,28,28,1)/255.0

In [ ]:
X.shape

建立資料的類別標籤，狗的類別標籤為 0，貓的類別標籤為 1。

In [ ]:
Y = np.zeros(2 * sample_size)

In [ ]:
Y[sample_size:] = 1.0

In [ ]:
Y.shape

在開始建立模型前，使用 Scikit-Learn 的 `train_test_split` 函式將 120000 筆資料順序隨機處理後，切分為訓練集和測試集。這邊設定的比例是訓練集 70%、測試集 30% 的切分比例。

切分後回傳得到訓練集資料、訓練集類別標籤、測試集資料、測試集類別標籤。

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, random_state=41, test_size=0.3)

In [3]:
print(train_x.shape) # (84000, 28, 28, 1)
print(test_x.shape) # (36000, 28, 28, 1)

NameError: name 'train_x' is not defined

類別標籤轉換為 one hot 編碼 (encoding)

In [4]:
train_y = tf.keras.utils.to_categorical(train_y)
test_y = tf.keras.utils.to_categorical(test_y)

NameError: name 'train_y' is not defined

## 3. 創建 CNN 類神經網路並訓練模型

在訓練模型前，須先定義並編譯 (compile) 模型。

下面定義了有 2 個卷積層 (`Conv2D()`)、2 個池化層 (`MaxPooling2D()`)、1 個平坦層 (`Flatten()`)、2 個全連接層 (`Dense()`) 的 CNN 網路。
 

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(), 
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2,activation='softmax')
])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.summary()

模型編譯部分，使用 Categorical Crossentropy 做為損失函數，優化器為 Adam，並以預測準確率 (accuracy) 做為模型表現在衡量指標。

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])

使用訓練集資料與訓練集類別標籤進行模型訓練，在訓練過程中並設定使用 20% 的資料做為驗證 (validation) 資料，進行交叉驗證。

In [ ]:
history = model.fit(x=train_x, y=train_y, batch_size=128, epochs=20, verbose=1, validation_split=0.2)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## 4. 測試與驗證準確率

訓練結束後，使用測試資料集與測試集類別標籤評估訓練後的模型。

In [ ]:
model.evaluate(test_x, test_y)

手動選擇一個圖片，查看預測結果是否正確。

執行後在輸入框輸入任一個圖片編號 ($\leq$ 35999，小於測試集的筆數中任一個數字)。

In [ ]:
test_no = int(input('please input photo number of test data: '))
plt.imshow(test_x[test_no].reshape(28,28))
print(f'it is {np.argmax(test_y[test_no])}')
pred = model.predict_classes(test_x[test_no:test_no+1])
print(f'predict it is {pred}')

## 5. 

In [ ]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(), 
  tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.01)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2,activation='softmax')
])

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
history = model2.fit(x=train_x, y=train_y, batch_size=128, epochs=20, verbose=1, validation_split=0.2)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()